In [5]:
import talib
# import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5

In [26]:
class BackTest:
    
    def __init__(self, instrument):
        
#         self.df = pd.DataFrame(yf.download(tickers=instrument, start = '2022-10-01', end = '2023-06-01', interval='1h'))
        if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
        login = 114781990
        password = 'Mypassword$1234'
        server = 'Exness-MT5Trial6'

        mt5.login(login, password, server)
        print('Working on: ', instrument[0])
        rate = mt5.copy_rates_from(instrument[0], mt5.TIMEFRAME_M15, datetime.datetime.now(), instrument[2])
        self.nf = pd.DataFrame(rate)
        
         
        self.nf['date']=pd.to_datetime(self.nf['time'], unit='s')

        '''Renaming columns and deleting unnecessary columns'''

        self.nf.set_index(np.arange(len(self.nf)), inplace = True)
        self.nf = self.nf.drop(['spread', 'real_volume', 'tick_volume'], axis='columns')
        
        
#         for trend
        self.position_index = 0
        
        # Calculate the number of rows in each DataFrame
        n = len(self.nf)
        n_half = n // 2  # Calculate the half point

        # Split the DataFrame into two equal parts
        self.df = self.nf.iloc[:n_half]  # First half
        self.df2 = self.nf.iloc[n_half:]  # Second half
        
        self.df.to_excel('check1.xlsx')
        self.df2.to_excel('check2.xlsx')

        
    def display(self, name):
        
        self.df.to_excel(f'{name}.xlsx')
        
        
    def define_trend(self, current_high, current_low, prev_high, prev_low, days, i):
        #       Uptrend
            if current_high > prev_high and current_low > prev_low:
    #           Find point at which current sample
    #           crossed prev high

                stream = self.df.loc[self.df.index[i : i+days], 'high']
                up_candle_index = stream.loc[stream > prev_high].index[0]
                
    #             if df.loc[i+15, 'high'] > prev_high:
                    
                self.df.loc[up_candle_index : i+days, 'trend'] = 'up'

            
                if self.df.iloc[i-1]['trend'] == 'up':
                    self.df.loc[i: up_candle_index, 'trend'] = 'up'

                else:
                    self.df.loc[i: up_candle_index, 'trend'] = 'no trend'


    #         Downtrend
            elif (current_high < prev_high) and (current_low < prev_low):
    #           Find point at which current sample 
    #           crossed prev low

                stream = self.df.loc[self.df.index[i : i+days], 'low']
                down_candle_index = stream.loc[stream < prev_low,].index[0]
               
#                 print('DownCandleIndex: ', down_candle_index)
#                 print('Days Callee: ', days)
                
                self.df.loc[down_candle_index : i+days, 'trend'] = 'down'

                if self.df.iloc[i-1]['trend'] == 'down':
                    self.df.loc[i: down_candle_index, 'trend'] = 'down'

                else:
                    self.df.loc[i: down_candle_index, 'trend'] = 'no trend'


                    
            else:
                self.df.loc[i : i+days, 'trend'] = 'no trend'

    
    def get_trend(self, i=0):
        
        length = len(self.df)

        # print('Length: ', length)
        
        days = 21

        prev_high = self.df.loc[self.df.index[i : i+days], 'high'].max()
        
        prev_low = self.df.loc[self.df.index[i : i+days], 'low'].min()

        while(i + days  < length):

            # print('In Trend !!')
        
            i += days
            
            current_high = self.df.loc[self.df.index[i : i+days], 'high'].max()
            current_low = self.df.loc[self.df.index[i : i+days], 'low'].min()
            
            self.define_trend(current_high, current_low, prev_high, prev_low, days, i)
            
            prev_high = current_high
            prev_low = current_low

            
                    
                
        if i < length-1:
            
            prev_high = self.df.loc[self.df.index[i-days:i], 'high'].max()
            prev_low = self.df.loc[self.df.index[i-days:i], 'low'].min()
            
            current_high = self.df.loc[self.df.index[i : length], 'high'].max()
            current_low = self.df.loc[self.df.index[i : length], 'low'].min()
            
            days = length-i
            
            self.define_trend(current_high, current_low, prev_high, prev_low, days, i)
            
#       Marking position of last candle with 21 group
        self.position_index = i
#         print('Past Position: ',self.position_index)
        self.prev_high = current_high
        self.prev_low = current_low
        
        
#    Calculating the current candle trend
    def test_current_trend(self, high, low, l):
        
#         print('High: ', self.prev_high)
        if high > self.prev_high and low > self.prev_low:

            self.df.loc[l, 'trend'] = 'up'

        elif high < self.prev_high and low < self.prev_low:

            self.df.loc[l, 'trend'] = 'down'

        else:
            self.df.loc[l, 'trend'] = self.df.loc[l-1,'trend']
        
    
#   Calculating the current candle trend
    def current_trend(self, days=21):
        l = len(self.df) - 1
        
#       check for trend in current candle

        if self.position_index + days > l:
            
            self.test_current_trend( self.df.iloc[l]['high'], self.df.iloc[l]['low'], l)
           
                
        elif self.position_index + days <= l:
            
            self.prev_high = self.df.loc[self.df.index[self.position_index : l+1], 'high'].max()
            self.prev_low = self.df.loc[self.df.index[self.position_index : l+1], 'low'].min()
            
            self.test_current_trend( self.df.iloc[l]['high'], self.df.iloc[l]['low'], l)
            
            
        #         Move position_index to 21 days window
        if self.position_index + days == l:
            
            self.position_index = l
            
            
    def get_current_data(self):
        
        for index, row in self.df2.iterrows():
            self.df = self.df.append(row, ignore_index=True)

            # Call the custom function after each append
            self.current_trend()
            

In [27]:
b = BackTest(['EURUSDm',10, 24000])

b.get_trend()
b.display('past')
b.get_current_data()
b.display('current')

Working on:  EURUSDm


C:\Users\hp\AppData\Local\Temp\ipykernel_8680\2075028449.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df.loc[down_candle_index : i+days, 'trend'] = 'down'
C:\Users\hp\AppData\Local\Temp\ipykernel_8680\2075028449.py:186: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)


In [28]:
current = pd.read_excel('./current.xlsx')
current.set_index(np.arange(len(current)), inplace = True)

current = current.dropna()

current.head()

,Unnamed: 0,time,open,high,low,close,date,trend
21,21,1663753500,0.99266,0.99372,0.99257,0.99305,2022-09-21 09:45:00,no trend
22,22,1663754400,0.99303,0.99303,0.99242,0.99268,2022-09-21 10:00:00,no trend
23,23,1663755300,0.99266,0.99344,0.99215,0.99313,2022-09-21 10:15:00,no trend
24,24,1663756200,0.99310,0.99310,0.99181,0.99247,2022-09-21 10:30:00,no trend
25,25,1663757100,0.99247,0.99296,0.99204,0.99213,2022-09-21 10:45:00,no trend


In [29]:
trend = pd.read_excel('check_trend.xlsx')

trend.head()

,sn,time,open,high,low,close,date,EMA_8,EMA_20,EMA_200,RSI,trend,squeeze_on,upper_keltner,lower_keltner
0,0,1663933500,0.97394,0.97572,0.97394,0.97508,2022-09-23 11:45:00,0.975345,0.975804,0.983937,38.304085,no trend,False,0.979523,0.972085
1,1,1663934400,0.97507,0.97602,0.97480,0.97536,2022-09-23 12:00:00,0.975348,0.975757,0.983851,39.450732,no trend,False,0.979370,0.972145
2,2,1663935300,0.97537,0.97683,0.97504,0.97633,2022-09-23 12:15:00,0.975566,0.975817,0.983776,43.284934,no trend,True,0.979475,0.972160
3,3,1663936200,0.97631,0.97651,0.97473,0.97558,2022-09-23 12:30:00,0.975569,0.975793,0.983695,41.168476,no trend,True,0.979488,0.972098
4,4,1663937100,0.97558,0.97586,0.97494,0.97553,2022-09-23 12:45:00,0.975561,0.975766,0.983614,41.028048,no trend,True,0.979287,0.972245


In [30]:
len(current)

23979

In [31]:
len(trend)

23780

In [33]:
print(current.loc[len(current)-1,'date'])

2023-09-05 23:00:00


In [34]:
print(trend.loc[len(trend)-1,'date'])

2023-09-06 04:30:00
